In [1]:
import sys
sys.path.append("..")
from MPRA_predict.utils import *
from MPRA_predict.datasets import *
from MPRA_predict.metrics import *

from collections import Counter
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 3)
np.set_printoptions(linewidth=1000)

In [ ]:
####### if model is enformer

# model_info_df = pd.read_csv('../pretrained_models/Enformer/Enformer_info.csv')
# predicted_df = np.load('outputs/SirajMPRA_Enformer_no_padding.npy').mean(1)
# predicted_df = np.load('outputs/SirajMPRA_Enformer_zero_padding.npy').mean(1)

# mask = (enformer_df['cell_type'] == cell_type) & (enformer_df['assay'].str.contains(assay))
#         index = enformer_df[mask].index.tolist()

In [10]:
assays = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27me3', 'H3K27ac', 'H3K36me3', 'CTCF']
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'A549', 'HCT116']

# # 定义细胞所有可能的名字
cell_names_dict = {
    'HepG2': [
        r"HepG2",
        r"Hep-G2",
        ],
    'K562': [
        r"K562",
        r"K-562",
    ],
    'SK-N-SH': [
        r"SK-N-SH",
        r"SKNSH",
        r"SKN-SH",
        r"SK-NSH",
        r"neuroblastoma",
    ],
    'A549': [
        r"A549",
        r"A-549",
    ],
    'HCT116': [
        r"HCT116",
        r"HCT-116",
    ],
}


for cell_type in cell_names_dict:
    cell_names = cell_names_dict[cell_type]
    pattern = re.compile("|".join(cell_names), re.IGNORECASE) # 不区分大小写
    matched_indice = [i for i in range(len(model_info_df))
                      if (pattern.search(model_info_df['cell_type'][i])) and (model_info_df['assay'][i] in assays)]

    counter = Counter(model_info_df.iloc[matched_indice]['cell_type'])
    for a in counter.most_common():
        print(a)
    print()

('HepG2_Epithelium_Liver', 42)
('HepG2_Hepatocellular_Carcinoma', 7)
('HepG2', 5)

('K562_Erythroblast_Bone_Marrow', 284)
('K562', 13)
('K562_Leukemia', 8)
('K562_Leukemia_Bone', 2)
('K562_human_erythroleukemic_cell_line', 1)

('Neuroblastoma_cell', 16)
('SHEP-21N_Neuroblastoma_cell_Bone_Marrow', 12)
('SK-N-SH_RA_Neuron_Brain', 9)
('SK-N-SH_Neuroblastoma_cell_Brain', 8)
('IMR-5_75_Neuroblastoma_cell', 4)
('SH-SY5Y_Neuroblastoma_cell', 4)
('Neuroblastoma_patient_cells', 3)
('SK-N-SH_RA', 3)
('BE2-C_Neuroblastoma_cell', 2)
('LAN-1_Neuroblastoma_cell', 2)
('CLB-Ga_Neuroblastoma_cell', 2)
('NB-1643_Neuroblastoma_cell', 1)
('SK-N-BE(2)c_Neuroblastoma_cell', 1)
('SJNB-8_Neuroblastoma_cell', 1)
('SK-N-DZ_Neuroblastoma_cell', 1)
('IMR-32_Neuroblastoma_cell', 1)
('SK-N-AS_Neuroblastoma_cell', 1)
('SJNB-6_Neuroblastoma_cell', 1)
('EBC-1_Neuroblastoma_cell', 1)
('KELLY_Neuroblastoma_cell', 1)
('CHP-212_Neuroblastoma_cell', 1)
('GI-CA-N_Neuroblastoma_cell', 1)
('SH-EP_Neuroblastoma_cell', 1)
('SJN

In [11]:
# # 定义细胞所有可能的名字
cell_names_dict = {
    'HepG2'  : ["HepG2_Epithelium_Liver", "HepG2_Hepatocellular_Carcinoma", "HepG2"],
    'K562'   : ["K562_Erythroblast_Bone_Marrow", "K562"],
    'SK-N-SH': ["SK-N-SH_RA_Neuron_Brain", "SK-N-SH_Neuroblastoma_cell_Brain", "SK-N-SH_RA"],
    'A549'   : ["A549_Epithelium_Lung", "A549"],
    'HCT116' : ["HCT-116_Colon", "HCT-116_HCT116", "HCT-116"],
}


cell_types = ['K562', 'HepG2', 'SK-N-SH', 'A549', 'HCT116']
assays = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27me3', 'H3K27ac', 'H3K36me3', 'CTCF']
index_df = pd.DataFrame(index=cell_types, columns=assays)

for cell_type in cell_types:
    for assay in assays:
        indice = []
        for cell_name in cell_names_dict[cell_type]:
            mask = (model_info_df['cell_type'].str.upper() == cell_name.upper()) & (model_info_df['assay'] == assay) # 不区分大小写
            index = model_info_df[mask].index.tolist()
            indice.extend(index)
        index_df.loc[cell_type, assay] = indice

index_df_len = index_df.map(len)
index_df_len

,DNase,H3K4me1,H3K4me3,H3K9me3,H3K27me3,H3K27ac,H3K36me3,CTCF
HepG2,9,4,8,2,2,9,3,17
K562,58,10,91,21,34,51,5,27
SK-N-SH,4,0,4,0,1,3,1,7
A549,4,37,45,27,3,53,1,10
HCT116,7,18,39,1,1,49,4,14


In [12]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'A549', 'HCT116']
assays = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27me3', 'H3K27ac', 'H3K36me3', 'CTCF']

feature_matrix = np.full(shape=(len(predicted_array), len(cell_types), len(assays)), fill_value=np.nan)
print(feature_matrix.shape)

(562654, 5, 8)


In [13]:
# average every channel

pearson_array = np.zeros((len(cell_types), len(assays)))
for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        indice = index_df.iloc[i, j]
        if indice:
            # predicted_feature = predicted_array[:, indice].mean(1)
            predicted_feature = logit(predicted_array[:, indice]).mean(1)
            true_label = MPRA_df[cell_type]
            r = pearson(predicted_feature, true_label)
            feature_matrix[:, i, j] = predicted_feature
            pearson_array[i, j] = r

print(pearson_array)
# print(feature_matrix.mean(-1))

[[ 0.44254926  0.35124665  0.46876764 -0.01119045  0.1974741   0.42731662  0.28557086  0.3502348 ]
 [ 0.49865128  0.38841764  0.45095817  0.28509025  0.18742642  0.44374257  0.32058434  0.36307116]
 [ 0.36796093  0.          0.46198413  0.          0.23614639  0.46956792  0.22345531  0.31575068]
 [ 0.34801156  0.30376153  0.36150358  0.23681118  0.08004334  0.36257705  0.26078315  0.24738493]
 [ 0.42579087  0.43542281  0.44558834 -0.05709654  0.2429226   0.45165317  0.2953102   0.33215694]]


In [11]:
# # if use less cell name
# cell_names_dict = {
#     'HepG2'  : ["HepG2_Epithelium_Liver"],
#     'K562'   : ["K562_Erythroblast_Bone_Marrow"],
#     'SK-N-SH': ["SK-N-SH_RA_Neuron_Brain", "SK-N-SH_Neuroblastoma_cell_Brain", "SK-N-SH_RA"],
#     'A549'   : ["A549_Epithelium_Lung"],
#     'HCT116' : ["HCT-116_Colon", "HCT-116_HCT116"],
# }


# cell_types = ['K562', 'HepG2', 'SK-N-SH', 'A549', 'HCT116']
# assays = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27me3', 'H3K27ac', 'H3K36me3', 'CTCF']
# index_list = []
# index_df = pd.DataFrame(index=cell_types, columns=assays)

# for cell_type in cell_types:
#     for assay in assays:
#         indice = []
#         for cell_name in cell_names_dict[cell_type]:
#             mask = (model_info_df['cell_type'].str.upper() == cell_name.upper()) & (model_info_df['assay'] == assay) # 不区分大小写
#             index = model_info_df[mask].index.tolist()
#             indice.extend(index)

#         index_df.loc[cell_type, assay] = indice
#         index_list.extend(indice)

# index_df_len = index_df.map(len)
# print(index_df_len)


# pearson_array = np.zeros((len(cell_types), len(assays)))
# for i, cell_type in enumerate(cell_types):
#     for j, assay in enumerate(assays):
#         indice = index_df.iloc[i, j]
#         if indice:
#             predicted_feature = logit(predicted_array[:, indice]).mean(1)
#             true_label = MPRA_df[cell_type]
#             r = pearson(predicted_feature, true_label)
#             feature_matrix[:, i, j] = predicted_feature
#             pearson_array[i, j] = r

# print(pearson_array)

In [14]:
np.save('Sei_Siraj_feature_matrix.npy', feature_matrix)